<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/colab/06_Asthma_Mortality_PD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 👨‍👩‍👧‍👦 Population Density (PD)

In this notebook we will  calculate the **population density (PD)** for each department in the study area from 2001 to 2022 based on our  annual population estimates and the area of each department.

##📦 Import Required Libraries

In [5]:
# geospatial data handling
import geopandas as gpd


from itables import init_notebook_mode

##🗂 Load Dataset

In [1]:
# change to my computer home directory
%cd work

/home/jovyan/work


In [4]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_2001_2022.gpkg")

In [ ]:
init_notebook_mode(all_interactive=True)
gdf.head()

## 🧮 Define Function to Calculate Population Density

In [12]:
def add_population_density(gdf):
    """
    Adds population density columns to a GeoDataFrame.

    Parameters:
    - gdf: GeoDataFrame in EPSG:4326 with population columns like 'A_2001', 'A_2002', ...

    Returns:
    - GeoDataFrame with new columns 'PD_2001', 'PD_2002', ... (people per km²)
    """
    # Project to EPSG:6933 (Equal Area projection)
    gdf_proj = gdf.to_crs(epsg=6933)

    # Calculate area in square kilometers
    gdf_proj["area_km2"] = gdf_proj.geometry.area / 1e6

    # Identify population columns
    pop_cols = [col for col in gdf.columns if col.startswith("A_") and col[2:].isdigit()]

    # Add population density columns
    for col in pop_cols:
        year = col.split("_")[1]
        gdf_proj[f"PD_{year}"] = round(gdf_proj[col] / gdf_proj["area_km2"], 2)

    # Drop intermediate area column if not needed
    gdf_proj = gdf_proj.drop(columns=["area_km2"])

    # Reproject back to EPSG:4326
    gdf_final = gdf_proj.to_crs(epsg=4326)

    return gdf_final

## ➕ Apply Population Density Function


In [13]:
# Add population density
gdf_with_density = add_population_density(gdf)

##👁 Preview Resulting GeoDataFrame and save results

In [ ]:
init_notebook_mode(all_interactive=True)
gdf_with_density.head()

In [15]:
# Save the GeoDataFrame to a GeoPackage file
gdf_with_density.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_2001_2022.gpkg", driver="GPKG")